In [1]:
import numpy as np
import pandas as pd

In [3]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [28]:
movies=movies.merge(credits,on='title')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4845 entries, 0 to 4844
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4845 non-null   int64  
 1   genres                4845 non-null   object 
 2   homepage              1719 non-null   object 
 3   id                    4845 non-null   int64  
 4   keywords              4845 non-null   object 
 5   original_language     4845 non-null   object 
 6   original_title        4845 non-null   object 
 7   overview              4842 non-null   object 
 8   popularity            4845 non-null   float64
 9   production_companies  4845 non-null   object 
 10  production_countries  4845 non-null   object 
 11  release_date          4844 non-null   object 
 12  revenue               4845 non-null   int64  
 13  runtime               4843 non-null   float64
 14  spoken_languages      4845 non-null   object 
 15  status               

In [32]:
# genres
# id
# keywords
# title
# overview
# cast
# crew
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [36]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [38]:
movies.dropna(inplace=True)

In [40]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [42]:
movies.duplicated().sum()

36

In [46]:
movies = movies.drop_duplicates()

In [48]:
movies.duplicated().sum()

0

In [52]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [54]:
movies['genres']=movies['genres'].apply(convert)

In [58]:
movies['keywords']=movies['keywords'].apply(convert)

In [63]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [65]:
movies['cast']=movies['cast'].apply(convert3)

In [72]:
def fetch_dir(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [74]:
movies['crew']=movies['crew'].apply(fetch_dir)

In [81]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [92]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [96]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [102]:
new_movies=movies[['movie_id','title','tags']]

In [104]:
new_movies

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4840,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4841,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4842,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4843,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [108]:
new_movies['tags']=new_movies['tags'].apply(lambda x:" ".join(x))

C:\Users\balbi\AppData\Local\Temp\ipykernel_3788\4113134046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags']=new_movies['tags'].apply(lambda x:" ".join(x))


In [122]:
new_movies['tags']=new_movies['tags'].apply(lambda x:x.lower())

C:\Users\balbi\AppData\Local\Temp\ipykernel_3788\2514494177.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags']=new_movies['tags'].apply(lambda x:x.lower())


In [143]:
import nltk

In [147]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [161]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [167]:
new_movies['tags']=new_movies['tags'].apply(stem)

C:\Users\balbi\AppData\Local\Temp\ipykernel_3788\2598606076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags']=new_movies['tags'].apply(stem)


In [169]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new_movies['tags']).toarray()

In [173]:
from sklearn.metrics.pairwise import cosine_similarity

In [177]:
similarity=cosine_similarity(vectors)

In [201]:
def recommend(movie):
    movie_index = new_movies[new_movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_movies.iloc[i[0]].title)

In [203]:
import pickle

In [205]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [207]:
pickle.dump(similarity,open('similarity.pkl','wb'))